<a href="https://www.kaggle.com/sardorabdirayimov/intro-to-titanic?scriptVersionId=84122521" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
%%time

df = pd.read_csv("/kaggle/input/titanic/train.csv")


CPU times: user 4.99 ms, sys: 1.86 ms, total: 6.85 ms
Wall time: 15.2 ms


In [3]:
sex_bin = {"female": 0,
           "male": 1}
df['Sex'] = df['Sex'].map(sex_bin)


In [4]:
import re 
def get_title(name):
    first = name.split(',')
    second = first[1].split('.')
    return second[0]


In [5]:
df.Name = df.Name.apply(get_title)

We REPLACED NAME TO TITLE:
* 17 UNIQUE TITLES
* TITLES ARE HELPFUL FOR PREDICTING AGE 
* TITLE HELPS TO PREDICT OTHER MISSING VALUES
* NOW I AM ANALYSING TITLES AND OTHER FEATURES


In [6]:
def best_age(name):
    return df.loc[(df.Name == name)].Age.mean()

WHAT IS DIFFERENCE BETWEEN MEAN & MEDIAN ?

In [7]:
df.Age=df.Age.fillna(df.Name.apply(best_age))

In [8]:
y_train = df.Survived
X_train = df.drop("Survived", axis=1)


In [9]:
X_train = X_train.drop(["Ticket", "Cabin"], axis=1)

In [10]:
X_train = X_train.drop(["PassengerId"], axis=1)

In [11]:
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,1,22.0,1,0,7.2500,S
1,1,Mrs,0,38.0,1,0,71.2833,C
2,3,Miss,0,26.0,0,0,7.9250,S
3,1,Mrs,0,35.0,1,0,53.1000,S
4,3,Mr,1,35.0,0,0,8.0500,S


In [12]:
X_train.Embarked.fillna("S", inplace=True)

In [13]:
features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True)

In [14]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,1
1,1,0,38.0,1,0,71.2833,0,0
2,3,0,26.0,0,0,7.9250,0,1
3,1,0,35.0,1,0,53.1000,0,1
4,3,1,35.0,0,0,8.0500,0,1


In [15]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [16]:
my_test = test.drop(["PassengerId","Cabin", "Ticket"],axis=1)

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [18]:
my_test.Name = my_test.Name.apply(get_title)

In [19]:
my_test.Age=my_test.Age.fillna(my_test.Name.apply(best_age))

In [20]:
my_test.Fare.fillna(test.Fare.mean(), inplace=True)

In [21]:
sex_bin = {"female": 0,
           "male": 1}
my_test['Sex'] = my_test['Sex'].map(sex_bin)

In [22]:
my_test2 = my_test.drop("Name", axis=1)

In [23]:
X_test = pd.get_dummies(my_test2, drop_first=True)

In [24]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,1,0
1,3,0,47.0,1,0,7.0000,0,1
2,2,1,62.0,0,0,9.6875,1,0
3,3,1,27.0,0,0,8.6625,0,1
4,3,0,22.0,1,1,12.2875,0,1


In [25]:
X.isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [26]:
from sklearn.ensemble import RandomForestClassifier 


In [27]:
model = RandomForestClassifier()


In [28]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
params = {"n_estimators": np.arange(50,700,50),
         "max_depth": np.arange(1,10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)


In [29]:
grid = GridSearchCV(estimator = model,
                   param_grid = params,
                   cv=skf,
                   n_jobs=-1,
                   verbose=1)
grid.fit(X, y_train)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 585 out of 585 | elapsed:  2.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=7, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650])},
             verbose=1)

In [30]:
grid.best_params_

{'max_depth': 8, 'n_estimators': 150}

In [31]:
grid.best_estimator_

RandomForestClassifier(max_depth=8, n_estimators=150)

In [32]:
grid.best_score_

0.8361182599962337

In [33]:
preds = grid.predict(X_test)

In [34]:
output = pd.DataFrame({"PassengerId": test.PassengerId,
                      "Survived": preds})
output.to_csv('My_random.csv', index=False)